Copyright 2021 The TensorFlow Similarity Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tensorflow Similarity Hello World

TensorFlow Similarity is a library for doing metric learning with TensorFlow. This can be used for tasks such as:

- finding similar items from within a collection of varied items
- semi-supervised or sparsely labeled contrastive learning

## Few-shot, contrastive learning

*Few-shot learning* is training a model based on a limited number of samples, rather than the very large number of well-labeled examples typically needed. In the context of TensorFlow means specifically that the trained model is able to make inferences about classes of examples it has not previously seen.

*Contrastive learning* is learning what features tend to make things similar or different. The goal is not to train a model that can identify which predefined class a new example is a member of, but rather a model which can compute the similarity between two new examples. This type of model is called a *similarity model*.

For example, a contrastive model trained on pictures of dogs might be able to tell you that a Saint Bernard and a Chihuahua are very different from each other, even if neither type of dog was present in the training set. Alternatively, a contrastive model trained on many different kinds of life (plants, animals, fungi), might find that the Saint Bernard and Chihuahua are much more similar to each than either is to a fern, even if no dogs at all were present in the training set.

## In this notebook

In this notebook you will:

1.  `train()` a similarity model on a sub-set of the 10 fashion MNIST classes that learn how to project images in a cosine space

2.  `index()` a small number of examples (10) of each of the classes present in the training dataset

3.  `lookup()` some the test data to show how our model is able to match unseen classes while having only a few examples of each in its index

4.  `calibrate()` the model to estimate what is the best distance theshold to separate matching elments from elements belonging to different classes

5.  `evaluate()` how well our model/index perform

6.  `match()` a few examples to test our calibration

## Imports and setup

In [ ]:
 %load_ext autoreload
 %autoreload 2

In [ ]:
from time import time
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Uncomment and run `!pip install...` if needed
# !pip install tensorflow_similarity

import tensorflow_similarity as tfsim  # main package
from tensorflow_similarity.utils import tf_cap_memory
from tensorflow_similarity.losses import TripletLoss  # specialized similarity loss
from tensorflow_similarity.layers import MetricEmbedding # layer with l2 regularization
from tensorflow_similarity.models import SimilarityModel # TF model with additional features
from tensorflow_similarity.samplers import MultiShotMemorySampler  # sample data 
from tensorflow_similarity.samplers import select_examples  # select n example per class
from tensorflow_similarity.visualization import viz_neigbors_imgs  # Neigboors vizualisation
from tensorflow_similarity.visualization import confusion_matrix  # matching performance

In [ ]:
tf_cap_memory()

In [ ]:
print('TensorFlow:', tf.__version__)
print('TensorFlow Similarity', tfsim.__version__)

# Data preparation

We are going to load the MNIST dataset and restrict our training data to only N of the 10 classes (6 by default) to showcase how the model is able to find similar examples from classes unseen during training which is one of the
main reason you would want to use metric learning.


**Important**: Tensorflow similarity expect `y_train` to be the examples class as integers. Accordingly contrary to a standard classification that do categorical encoding we are leavig `y_train` as is.

In [ ]:
NUM_CLASSES = 6  #@param {type: "slider", min: 1, max: 10}
classes = [2, 3, 1, 7, 9, 6, 8, 5, 0, 4]
# note we added a 11th classes for unknown
labels = ["0", "1",  "2", "3", "4", "5", "6", "7", "8", "9", "Unknown"]
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data('/tmp/data/mnist.npz')

x_train = tf.constant(x_train / 255.0, dtype='float32')
x_test = tf.constant(x_test / 255.0, dtype='float32')

x_restricted, y_restricted = select_examples(x_train, y_train, classes[:NUM_CLASSES], 30000)
print(x_restricted.shape, y_restricted.shape)

To learn efficiently each batch must contains at least 2 sample of each class. Additionally it is often useful to have multiple examples of each class so we can mine informative triplet using different mining strategies (e.g *semi-hard* mining).

To make this easy tf_similarity offers `Samplers()` that allows to control the number of classes and minimal ammount of examples in each class. Here we are going `MultiShotMemorySampler()` which allows to sample a dataset that reside in memory and have muliples examples per class.

In [ ]:
CLASS_PER_BATCH = NUM_CLASSES #@param {type:"integer"}
EXAMPLE_PER_CLASS = 6 #@param {type:"integer"}
STEPS_PER_EPOCH = 1000 #@param {type:"integer"}
sampler = MultiShotMemorySampler(x_restricted, y_restricted, class_per_batch=CLASS_PER_BATCH, example_per_class=EXAMPLE_PER_CLASS, steps_per_epoch=STEPS_PER_EPOCH)

# model setup

## Model definition

Triplet loss models are normal `tensorflow.keras` models except their output is a distance embeding. We use `SimilarityModel()` instead of the default `Model()` because this subclass contains additional proprerty that makes indexing and searching embeddings easy once the training is done.

In [ ]:
def get_model():
    inputs = tf.keras.layers.Input(shape=(28,28))
    x = layers.Reshape((28, 28, 1))(inputs)
    x = layers.Conv2D(32, 7, activation='relu')(x)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 7, activation='relu')(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    # dont make the embedding to large - its slow down the lookups
    outputs = MetricEmbedding(32)(x)
    return SimilarityModel(inputs, outputs)
model = get_model()
model.summary()

## Loss definition

Overall Similarity losses (aka metric losses) are similar to other losses except they expect different inputs.

For triplet loss, one of the most popular one, you need to decide what type of triplet mining you want. Usually `hard` mining for positive example and `semi-hard` mining work well and are the default. They are passed explicitly here for reference.

The `distance` parameters specify which distance metric to use to compute the distance between embeddings. `cosine` is usually a great starting point and the default.


In [ ]:
distance = 'cosine' #@param ["cosine"]{allow-input: false}
positive_mining_strategy = 'hard' #@param ["easy", "hard"]{allow-input: false}
negative_mining_strategy = 'semi-hard'  #@param ["easy", "semi-hard", "hard"]{allow-input: false}
triplet_loss = TripletLoss(distance=distance,
    positive_mining_strategy=positive_mining_strategy,
    negative_mining_strategy=negative_mining_strategy)

## Compilation

Tensorflow similarity use an extended `compile()` method that allows you to specify distance_metrics (metrics that are computed over the distance between the embedding). 

By default the `compile()` method try to infers what type of distance you are using by looking at the fist loss specified. If you use multiples losses and the distance loss is not the first, then you need to specify the distance function used as `distance=` parameter in the compile function.

In [ ]:
LR = 0.001  #@param {type:"float"}
model = get_model()
model.compile(optimizer=Adam(LR), loss=triplet_loss)

# Training

Similarity are trained like normal model. Don't expect the validation loss to decrease too much because we only use half of the class on the train data but all class in the validation.

In [ ]:
EPOCHS = 5 #@param {type:"integer"}
history = model.fit(sampler, validation_data=(x_test, y_test), epochs=EPOCHS)

In [ ]:
# expect loss: 0.07 / val_loss: 0.6
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('Loss')
plt.show()

# Indexing

Indexing is where things get different from traditional classification models. Because the model learned to output an embedding that represent the example position in a metric space and not a classification, we need a way to find which known example(s) are the closest to determine the class of the incoming data (aka nearest neighboors classication).

To do so we need to **create an index of known example from the 10 cloth classes**. We do this by taking **200 examples from the train dataset which amount to 20 examples** for each class and use the `index()` funtion of the model to build the index.

we store the images (x_index) as data in the index (`data=x_index`) so we can display them later. Here the images are small so its not an issue but in general, be careful while storing a lot of data in the index to avoid blewing up your memory. You might consider using a different `Table()` backend if you have to store and serve very large indexes.

Adding more examples per class, helps increase the accuracy/generalization as you improve the set of known variations that the classifier "knows". 

Reseting the index is not necessary but it ensure we start with a clean index in case of a re-run

In [ ]:
x_index, y_index = select_examples(x_train, y_train, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 20)
model.reset_index()
model.index(x_index, y_index, data=x_index)

# Querying

To "classify" our testing data, we need for each testing example to look for its *k* [nearest neighbors](https://scikit-learn.org/stable/modules/neighbors.html) in the index.

We query a single random example for each class from the test dataset using  `select_examples()` and then find their nearest neighboors using the `single_lookup()` function.

In [ ]:
# re-run to test on other examples
num_neighboors = 5

# select
x_display, y_display = select_examples(x_test, y_test, list(range(10)), 1)

# lookup
nns = model.lookup(x_display, k=num_neighboors)

# display
for idx in np.argsort(y_display):
    viz_neigbors_imgs(x_display[idx], y_display[idx], nns[idx], 
                      labels=labels, fig_size=(16, 2), cmap='Greys')

# matching

To be able to tell if an example match a given class, we first need to `calibrate()` the model to know what is the optimal cut point. By default, we only compute an optimal cutpoint that maximizes performance. However you can speficy your own and change the optimization metrics as illustrated in later notebooks/colabs

In [ ]:
num_calibration_samples = 1000
calibration = model.calibrate(x_train[:num_calibration_samples], y_train[:num_calibration_samples], verbose=1)

Let's look how the performance metrics evolve as the distance between the sample increase. We clearly see the F1 optimial cutpoint where the precision and recall intersect which what the `optimal_cutpoint` represent.

In [ ]:
fig, ax = plt.subplots()
ax.plot(calibration['thresholds']['accuracy'], label='precision')
ax.plot(calibration['thresholds']['recall'], label='recall')
ax.plot(calibration['thresholds']['f1_score'], label='f1 score')
ax.legend()
ax.set_title("Metric evolution as distance increase")
ax.set_xlabel('Examples')
plt.show()

Ploting the precision/recall curve we can see that for a lot of samples from the unseen classes the model is not doing really well (you see the inflexion past 0.6 recall). This can be improved by indexing more data, using data augmentation, a better model or even changing which classes are trained on. 

In [ ]:
fig, ax = plt.subplots()
ax.plot(calibration['thresholds']['recall'], calibration['thresholds']['accuracy'])
ax.set_title("Precision recall curve")
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
plt.show()

# Evaluation
We evaluate our model ability to accurately match new examples using our calibrated threshold by testing how many examples from the test sets are correctly matched. The example which have a distance above the cutoff are excluded and labeled as unknown.

The evaluation below reports 

In [ ]:
metrics = model.evaluate_matching(x_test, y_test)

# match

Let's now match a 100 examples to see how you can use the `match()` function in practice. `match()` return you what class the example belong to based of the content of the index and allows to use your model to make prediction on an unbounded, added after training, classes!

Note: `match()` return -1 by default when the distance between the submited example and any point of the index is above the distance cutpoint.

In [ ]:
num_matches = 10

matches = model.match(x_test[:num_matches], cutpoint='optimal')
rows = []
for idx, match in enumerate(matches):
    rows.append([match, y_test[idx], match == y_test[idx]])
print(tabulate(rows, headers=['Predicted', 'True', 'correct']))

## confusion matrix
Let's select a few hundreds samples for each class in the test set to plot the confusion matrix for a few

**note** `tf.math.confusion_matrix` doesn't support negative class so we are going to use **10 as our unknown class**

In [ ]:
num_example_per_class = 1000
cutpoint = 'optimal'

x_confusion, y_confusion = select_examples(x_test, y_test, range(10), num_example_per_class)

matches = model.match(x_confusion, cutpoint=cutpoint, no_match_label=10)
confusion_matrix(matches, y_confusion, labels=labels, title='Confusin matrix for cutpoint:%s' % cutpoint)

# Index information

Following `model.summary()` you can get information about the index configuration and its performance using `index_summary()`.

In [ ]:
model.index_summary()

# Saving and reloading
Saving and reloading the model works as you would expected: 
- `model.save(path)`: save the model and the index on disk (`save_index=False`). By default the index is compressed - can be disabled by setting `compressed=False`
- `model = tf.keras.model.load_model(path)` reload the model. To reload the index you then need to call model.load_index(path)
- `model.save_index(path)` and `model.load_index(path)` allows to save/reload an index indenpendly of saving/loading a model if need to be


## Saving

In [ ]:
# save the model and the index
save_path = 'models/hello_world' #@param {type:"string"}
model.save(save_path)

## Reloading

In [ ]:
# reload the model
reloaded_model = load_model(save_path)
# reload the index
reloaded_model.load_index(save_path)

In [ ]:
#check the index is back
reloaded_model.index_summary()

## Query reloaded model
Querying the reloaded model with its reload index works as expected

In [ ]:
# re-run to test on other examples
num_neighboors = 5

# select
x_display, y_display = select_examples(x_test, y_test, list(range(10)), 1)

# lookup
nns = model.lookup(x_display, k=num_neighboors)

# display
for idx in np.argsort(y_display):
    viz_neigbors_imgs(x_display[idx], y_display[idx], nns[idx], 
                      labels=labels, fig_size=(16, 2), cmap='Greys')